# Programación declarativa @ URJC
# Programación funcional
## Curso 22-23, convocatoria ordinaria (31 de octubre de 2022)
## Campus de Vicálvaro


# Definiciones auxiliares

In [17]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [18]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._


defined function foldTree

In [19]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard y Tipos Algebraicos de Datos

In [20]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A
}

defined trait Isomorphic

In [21]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la validez del siguiente esquema proposicional: 

$\vdash \neg \neg (p \vee \neg p)$

In [22]:
def proof[P]: Not[Not[Either[P, Not[P]]]] =
    (ne: Either[P, Not[P]] => Nothing) =>
        ne(Right((p: P) => ne(Left(p))))

defined function proof

__b) (1 punto)__ Utiliza la correspondencia Curry-Howard para demostrar la validez del siguiente esquema proposicional: $p \wedge \neg p \vdash q$, __sin__ utilizar la regla de inferencia según la cual del absurdo, todo se sigue, es decir: $\bot \vdash q$. Para ello, utiliza como premisa adicional la ley de la doble negación aplicada a la proposición $q$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ \neg \neg q \rightarrow q \} \vdash (p \wedge \neg p) \rightarrow q$

In [23]:
def proof[P, Q](dn: Not[Not[Q]] => Q): (P, Not[P]) => Q = 
    (p, np) => dn(nq => np(p))

defined function proof

# Ejercicio 2
__(1 punto)__

Demuestra que, para todo tipo `A` y `R`, los tipos de datos algebraicos $R^A*R^1$ y $R^{A+1}$ son isomorfos. Utiliza el tipo de datos `Option[A]` para representar el tipo algebraico $A+1$.

In [24]:
class Iso[A, R] extends Isomorphic[(A => R, Unit => R), Option[A] => R]{
    
    def from(f: (A => R, Unit => R)): Option[A] => R = 
        {
            case None => f._2()
            case Some(a) => f._1(a)
        }

    def to(f: Option[A] => R): (A => R, Unit => R) = 
        (a => f(Some(a)), _ => f(None))
}

defined class Iso

# Ejercicio 3
__(3 puntos)__

Dado el siguiente tipo de datos recursivo:

In [73]:
case class Struct[A](elements: List[Either[A, Struct[A]]])

defined class Struct

implementa una función `flatten: List[Either[A, Struct[A]]] => List[A]`, que permita extraer todos los elementos de la estructura en una nueva lista. Ténganse en cuenta las siguientes pruebas:

In [78]:
class TestFlatten(flatten: List[Either[Int, Struct[Int]]] => List[Int])
extends FlatSpec with Matchers{
    
    val l: List[Either[Int, Struct[Int]]] = 
        List(Left(1), 
             Left(2), 
             Right(Struct(List(Left(3), 
                               Right(Struct(List())),
                               Left(4),
                               Right(Struct(List(Left(5))))))))
    
    "flatten" should "work" in {
        flatten(l) shouldBe List(1,2,3,4,5)
    }
}

defined class TestFlatten

__a) (1 punto)__ Implementa la función `flatten` de manera recursiva:

In [79]:
def flatten[A](l: List[Either[A, Struct[A]]]): List[A] = 
    l match {
        case Nil => Nil
        case Left(a) :: tail => a :: flatten(tail)
        case Right(Struct(l)) :: tail => flatten(l) ++ flatten(tail)
    }

defined function flatten

In [80]:
run(new TestFlatten(flatten))

cmd77$Helper$TestFlatten:
flatten
- should work


__b) (1 punto)__ Implementa la función `flatten` utilizando la función de orden superior `foldRight`:

In [82]:
def flatten[A](l: List[Either[A, Struct[A]]]): List[A] = 
    l.foldRight(Nil: List[A]){
        case (Left(a), tailSol) => a :: tailSol
        case (Right(Struct(h)), tailSol) => 
            flatten(h) ++ tailSol
    }

defined function flatten

In [83]:
run(new TestFlatten(flatten))

cmd77$Helper$TestFlatten:
flatten
- should work


__c) (1 punto)__  Implementa la función `flatten` utilizando la función de orden superior `foldLeft`:

In [84]:
def flatten[A](l: List[Either[A, Struct[A]]]): List[A] = 
    l.foldLeft(Nil: List[A]){
        case (acc, Left(a)) => acc :+ a
        case (acc, Right(Struct(h))) => acc ++ flatten(h) 
    }

defined function flatten

In [85]:
run(new TestFlatten(flatten))

cmd77$Helper$TestFlatten:
flatten
- should work


# Ejercicio 4
__(3 puntos)__

Considérese la siguiente definición de árbol binario:

In [36]:
sealed trait Tree[A]
case class Leaf[A](a: A) extends Tree[A]
case class Branch[A](l: Tree[A], r: Tree[A]) extends Tree[A]

defined trait Tree
defined class Leaf
defined class Branch

__a) (1 punto)__ Implementa una función `isEven: Tree[Int] => Boolean` que dado un árbol binario de número enteros, devuelve cierto si el número de elementos pares es par, y falso en caso contrario. Por ejemplo:

In [52]:
class TestIsEven(isEven: Tree[Int] => Boolean) extends FlatSpec with Matchers{
    
    val t1: Tree[Int] = 
        Branch(Branch(Leaf(1), Branch(Branch(Leaf(2), Leaf(3)), Leaf(4))), Leaf(5))
    
    val t2: Tree[Int] = 
        Branch(Branch(Leaf(0), Branch(Branch(Leaf(2), Leaf(3)), Leaf(4))), Leaf(5))
    
    "isEven" should "work" in {
        isEven(t1) shouldBe true
        isEven(t2) shouldBe false
        isEven(Leaf(0)) shouldBe false
        isEven(Leaf(1)) shouldBe true
    }
}

defined class TestIsEven

In [53]:
def isEven(t: Tree[Int]): Boolean = 
    t match {
        case Leaf(n) => n % 2 != 0
        case Branch(l, r) => isEven(l) && isEven(r) || !isEven(l) && !isEven(r)
    }

defined function isEven

In [54]:
run(new TestIsEven(isEven))

cmd51$Helper$TestIsEven:
isEven
- should work


__b) (1 punto)__ Implementa la función `total` que dado un árbol de cadenas de caracteres, devuelve la suma de todas las longitudes de las cadenas almacenadas en el árbol. Por ejemplo:

In [64]:
class TestTotal(total: Tree[String] => Int) extends FlatSpec with Matchers{
    
    val t1: Tree[String] = 
        Branch(Branch(Leaf("hi"), Branch(Branch(Leaf("lucas"), Leaf("eh")), Leaf("oh"))), Leaf("w"))
    
    "total" should "work" in {
        total(t1) shouldBe 12
        total(Leaf("")) shouldBe 0
        total(Leaf("a")) shouldBe 1
    }
}

defined class TestTotal

In [65]:
def total(t: Tree[String]): Int = 
    t match {
        case Leaf(s) => s.length
        case Branch(l, r) => total(l) + total(r)
    }

defined function total

In [66]:
run(new TestLength(total))

cmd54$Helper$TestLength:
length
- should work


__c) (2 puntos)__ Implementa una función de orden superior `fold`, similar la función `foldTree` explicada en clase, que permita reimplementar de una forma más modular las dos funciones anteriores. 

In [67]:
def fold[A, B](t: Tree[A])(leaf: A => B)(branch: (B, B) => B): B = 
    t match {
        case Leaf(a) => leaf(a)
        case Branch(l, r) => branch(fold(l)(leaf)(branch), fold(r)(leaf)(branch))
    }

defined function fold

Reimplementa las funciones `isEven` y `total` utilizando la función de orden superior `fold`:

In [68]:
def isEven(t: Tree[Int]): Boolean = 
    fold(t)(_ % 2 != 0)((el, er) => el && er || !el && !er)

defined function isEven

In [69]:
run(new TestIsEven(isEven))

cmd51$Helper$TestIsEven:
isEven
- should work


In [70]:
def total(t: Tree[String]): Int = 
    fold(t)(_.length)(_ + _)

defined function total

In [72]:
run(new TestTotal(total))

cmd63$Helper$TestTotal:
total
- should work
